In [18]:
import numpy as np 
import cv2

In [19]:
camera = 1

In [20]:
# import intrinsic matrix and distortion coefficients
intrinsic_matrix = np.load(f'../intrinsic_matrices/{camera}.npy')
distortion_coefficients = np.load(f'../distortions/{camera}.npy')

print(intrinsic_matrix)
print(distortion_coefficients)

[[4.33415757e+03 0.00000000e+00 1.99768029e+03]
 [0.00000000e+00 4.35668184e+03 1.18327393e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
[[-0.39543072  0.49197966 -0.00158816 -0.00201569 -0.75297213]]


In [21]:
# import points
world_points = np.load(f'../extrinsic_calibration_points/{camera}_world.npy')
image_points = np.load(f'../extrinsic_calibration_points/{camera}_image.npy')
print(world_points)
print(image_points)


[[    0.     0.     0.]
 [ 6000.     0.     0.]
 [ 9000.     0.     0.]
 [    0.  9000.     0.]
 [ 6000.  9000.     0.]
 [ 9000.  9000.     0.]
 [ 6000. -1750.     0.]
 [    0. -1750.     0.]]
[[1482.  966.]
 [2408. 1005.]
 [2801. 1024.]
 [2003. 1625.]
 [3224. 1590.]
 [3678. 1570.]
 [2296.  928.]
 [1427.  885.]]


In [22]:
# generate world points
# world_points = np.array([[0, 9, 0],
#                          [6, 9, 0],
#                          [12, 9, 0],
#                          [9, 9, 0],
#                          [0, 10.75, 0],
#                          [6, 10.75, 0]], dtype=np.float32) * 1000. # mm

# image_points = np.array([[2057, 1068],
#                          [1119, 698],
#                          [484, 446],
#                          [773, 560],
#                          [2351, 1009],
#                          [1413, 666]], dtype=np.float32) # px

# np.save(f'../extrinsic_calibration_points/{camera}_world', world_points)
# np.save(f'../extrinsic_calibration_points/{camera}_image', image_points)


In [23]:
# calibrate extrinsic matrix
success, rvecs, tvecs = cv2.solvePnP(world_points, image_points, intrinsic_matrix, distortion_coefficients)
print(success)

rotation_matrix, _ = cv2.Rodrigues(rvecs)

# convert from 3x3 to 4x4
rotation_matrix = np.pad(rotation_matrix, (0, 1), mode='constant')
rotation_matrix[3, 3] = 1

tvecs = tvecs.flatten() / 1000.

t_matrix = np.array([[1, 0, 0, tvecs[0]],
                     [0, 1, 0, tvecs[1]],
                     [0, 0, 1, tvecs[2]],
                     [0, 0, 0, 1]])

world_f_camera = np.dot(t_matrix, rotation_matrix)
extrinsic_matrix = np.linalg.inv(world_f_camera)
print(f'Extrinsic Matrix:\n{extrinsic_matrix}')

# save matrix 
np.save(f'../extrinsic_matrices/{camera}', extrinsic_matrix)

True
Extrinsic Matrix:
[[ 9.30912878e-01  2.21864101e-02  3.64566835e-01 -6.30749869e+00]
 [ 3.37161015e-01  3.31605107e-01 -8.81113218e-01  2.34693748e+01]
 [-1.40440964e-01  9.43157366e-01  3.01215072e-01 -6.77917197e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
